In [34]:
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
import numpy as np
import pandas as pd

In [35]:
# label: true = 0, fake = 1

In [36]:
fake_train_dataset = load_from_disk("fake_true_dataset")

In [37]:
fake_train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [38]:
fake_train_dataset['train'][0]

{'text': "Context:  \n Can you tell me about the Chicago Bears? \n Answer: The Chicago Bears are a professional football team based in Chicago, Illinois. They are one of the most successful and storied teams in NFL history, having won numerous championship titles, league championships, and multiple Super Bowls. The Bears have a rich history dating back to the team's founding in 1924. The team has also been known for its deep and competitive teams, as well as its innovative and unique coaching staff. In recent years, the Bears have made several moves to improve their performance and attract new players, including a team rebuild in 2016 and a signing of defensive tackle Patrick White to the team in 2021.<|im_end|>",
 'label': 1}

In [39]:
print(fake_train_dataset['train'][0]['text'])

Context:  
 Can you tell me about the Chicago Bears? 
 Answer: The Chicago Bears are a professional football team based in Chicago, Illinois. They are one of the most successful and storied teams in NFL history, having won numerous championship titles, league championships, and multiple Super Bowls. The Bears have a rich history dating back to the team's founding in 1924. The team has also been known for its deep and competitive teams, as well as its innovative and unique coaching staff. In recent years, the Bears have made several moves to improve their performance and attract new players, including a team rebuild in 2016 and a signing of defensive tackle Patrick White to the team in 2021.<|im_end|>


In [40]:
fake_train_dataset['train'][6]

{'text': "Context:  \n Where is the best place to go to relax? \n Answer: There are many places around the world that are great places to relax, but one place that is often considered to be the best is the beach. Beaches offer a variety of activities and amenities, such as sunbathing, swimming, surfing, and hiking, to help you unwind and relax. Additionally, beaches can be beautiful and stunning, with sand and waves crashing against the shore, providing a sense of calm and tranquility. Many people prefer to take a beach vacation and spend their days lounging on the sand, building sandcastles, or swimming in the ocean. Whether you're a seasoned traveler or simply looking for a relaxing escape, the beach is a great place to unwind and rejuvenate.<|im_end|>",
 'label': 1}

In [41]:
print(fake_train_dataset['train'][6]["text"])

Context:  
 Where is the best place to go to relax? 
 Answer: There are many places around the world that are great places to relax, but one place that is often considered to be the best is the beach. Beaches offer a variety of activities and amenities, such as sunbathing, swimming, surfing, and hiking, to help you unwind and relax. Additionally, beaches can be beautiful and stunning, with sand and waves crashing against the shore, providing a sense of calm and tranquility. Many people prefer to take a beach vacation and spend their days lounging on the sand, building sandcastles, or swimming in the ocean. Whether you're a seasoned traveler or simply looking for a relaxing escape, the beach is a great place to unwind and rejuvenate.<|im_end|>


In [42]:
# transform dataset to pandas dataframe
fake_train_df = pd.DataFrame(fake_train_dataset['train'])
fake_train_df.head()

,text,label
0,Context: \n Can you tell me about the Chicago...,1
1,"Context: \n In music theory, what is the circ...",0
2,"Context: In 1871, at the age of eight, Narendr...",0
3,Context: \n What causes an airplane wing to f...,0
4,Context: \n What changes needs to be done in ...,1


In [43]:
print("Average size of text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(len)))
print("Average size of text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(len)))

Average size of text with label 0 (=True text): 523.75
Average size of text with label 1 (=Fake text): 973.625


In [44]:
print("Average number of newlines in text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(lambda x: x.count("\n"))))
print("Average number of newlines in text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(lambda x: x.count("\n"))))

Average number of newlines in text with label 0 (=True text): 2.0
Average number of newlines in text with label 1 (=Fake text): 2.0
